<a href="https://colab.research.google.com/github/MicroprocessorX069/Collaborative-Filtering-for-medical-history/blob/master/CF_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df=pd.read_csv("dummy.csv",header=None)

In [0]:
import math
#adjacent elements of original dataset added here in df_adj
df_adj=pd.DataFrame()
for x in range(len(df)):
  y=0
  while y+1<len(df.iloc[x]) and not(pd.isna(df.iloc[x,y+1])):
    temp=df.iloc[x,y]
    temp2=df.iloc[x,y+1]
    df2 = pd.DataFrame([[temp,temp2]])
    df_adj=df_adj.append(df2)
    y+=1
#df_adj=df_adj.reset_index() # to make the indices of rows as 0,1,2,3...

In [0]:
df_adj2=pd.get_dummies(df_adj)
processedData=df_adj2[['0_a',	'0_b',	'0_c',	'0_d',	'0_e'	]]
processedLabel=df_adj2[['1_a',	'1_b',	'1_c',	'1_d',	'1_e'	]]

In [0]:
import numpy as np
processedData=np.array(processedData)
processedLabel=np.array(processedLabel)


In [0]:
#@title Parameters for the model { run: "auto" } 

import tensorflow as tf
#run:auto command makes the code block run in real time. Whenever the user amends the input, the block runs automatically for the new set of input.
no_of_layers= 3 #@param {type:"slider", min:0, max:10, step:1}
# A is the list of nodes_in_each_hidden_layer for each of the user defined hidden layers. Eg. A=[128,256,64] implies 3 hidden layers with 128 nodes, 256 nodes and 64 nodes respectively starting from the input layer to the output layer.
A=[]
# No. of nodes in hidden layer 1
nodes_in_layer1 = 8 #@param {type:"integer"}
A.append(nodes_in_layer1)
#dropout_rate = 0.1 #@param {min:0.000,max:1.000}

nodes_in_layer2 = 8 #@param {type:"integer"}
A.append(nodes_in_layer2)
nodes_in_layer3 = 8 #@param {type:"integer"}
A.append(nodes_in_layer3)
validation_splitt=0 #@param{type:"slider",min:0.000,max:1.000,step:0.05}
Activation_function = tf.nn.crelu #@param ["relu", "relu6", "crelu", "elu", "selu", "softplus", "softsign", "dropout", "bias_add", "sigmoid", "tanh"]
loss_function = 'kullback_leibler_divergence' #@param ["categorical_crossentropy", "sparse_categorical_crossentropy", "binary_crossentropy","kullback_leibler_divergence","poisson","cosine_proximity"]
optimizer_used = 'SGD' #@param ["SGD", "RMSprop", "Adagrad","Adadelta","Adam","Adamax","Nadam","TFOptimizer"]

In [48]:
#@title Parametres for the model { run: "auto" }

#no_epochs is the integer input for total no. of model parsing -
#from input to generating output and back to updating the weights -
#according to the error for all training examples.

no_epochs=1000 #@param{type:"slider",min:0,max:10000,step:200}
 
import time
start=time.time()

#Libraries for current codeblock
import tensorflow as tf
from tensorflow import nn
nodes_in_output_layer=5
input_size=5

model=tf.keras.models.Sequential()

#Various types of layers in keras: Dense, Flatten, Input, Permute, Reshape.[3]
#Dense implements the operation: output = activation(dot(input, kernel) + bias)[3]

#Adding input layer with size of input i.e. 10 in this case. 
#The first parameter for function Dense() is No. of nodes. in the layer. model.add appends the newly created hidden layer to the model.

model.add(tf.keras.layers.Dense(A[0],activation=Activation_function,input_dim=input_size))
for layer_iterator in range(1,len(A)-1):
  model.add(tf.keras.layers.Dense(A[layer_iterator+1], activation=Activation_function))
#Creating dropout layer
#model.add(tf.keras.layers.Dropout(rate=dropout_rate))
# Creating the output layer with 4 nodes. Since Output layer 
model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))

# Parameters for model creation i.e. optimizers, loss function
                                                                                # metric: Accuracy. Accuracy is the measure of correctness of the model
                                                                                # - , basically rate to which it classifies the inputs correctly.
                                                                                # Validation Accuracy: Percentage of correctly classifying on input chosen from the train dataset itself.
                                                                                # Testing Accuracy:  Percentage of correctly classifying on input from a new dataset following the same problem as the train dataset.

                                                                                # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
model.summary()
history=model.fit(processedData, processedLabel, epochs= no_epochs)

end=time.time()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 16)                48        
_________________________________________________________________
dense_22 (Dense)             (None, 16)                136       
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 85        
Total params: 269
Trainable params: 269
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
34/34 [==============================] - 0s 8ms/step - loss: 1.5492 - acc: 0.2059
Epoch 2/1000
34/34 [==============================] - 0s 308us/step - loss: 1.5472 - acc: 0.2059
Epoch 3/1000
34/34 [==============================] - 0s 448us/step - loss: 1.5465 - acc: 0.2059
Epoch 4/1000
34/34 [==============================] - 0s 398us/step - loss: 1.5445 - acc: 0.2059
Epoch 5/1000
34/34 [==

In [0]:
# loss,acc=model.evaluate(processedData,processedLabel)
# print("Loss=",loss,"\nAccuracy=",acc)
# time_taken=end-start
# print("Time Taken=",end-start)
# %matplotlib inline
# df = pd.DataFrame(history.history)
# df.plot(subplots=True, grid=True, figsize=(10,15))
# print(np.std(df))

In [55]:
predictedTestLabel = []
for i,j in zip(processedData,processedLabel):
    y = model.predict(np.array(i).reshape(-1,5))
    predictedTestLabel.append(y.argmax())
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "a"
    elif encodedLabel == 1:
        return "b"
    elif encodedLabel == 2:
        return "c"
    elif encodedLabel == 3:
        return "d"
    elif encodedLabel == 4:
        return "e"

predictedTestLabel2=[]
for Label in predictedTestLabel:
  predictedTestLabel2.append(decodeLabel(Label))
predictedTestLabel2=pd.DataFrame(predictedTestLabel2)
print(pd.concat([df_adj,predictedTestLabel2],ignore_index=True))

    0    1
0   a    b
1   b    c
2   c    a
3   a    c
4   c    b
5   b    c
6   c    c
7   c    b
8   a    b
9   b    b
10  b    a
11  a    a
12  a    c
13  c    c
14  c    a
15  a    a
16  a    b
17  b    c
18  c    b
19  b    a
20  c    a
21  a    d
22  d    a
23  a    b
24  b    e
25  e    a
26  a    b
27  b    a
28  a    e
29  e    e
.. ..  ...
38  b  NaN
39  a  NaN
40  b  NaN
41  b  NaN
42  b  NaN
43  a  NaN
44  a  NaN
45  b  NaN
46  b  NaN
47  b  NaN
48  b  NaN
49  b  NaN
50  b  NaN
51  a  NaN
52  b  NaN
53  a  NaN
54  b  NaN
55  b  NaN
56  a  NaN
57  b  NaN
58  a  NaN
59  a  NaN
60  b  NaN
61  a  NaN
62  b  NaN
63  a  NaN
64  a  NaN
65  b  NaN
66  a  NaN
67  a  NaN

[68 rows x 2 columns]


In [52]:
pd.DataFrame(predictedTestLabel2)

,0
0,b
1,a
2,b
3,b
4,b
5,a
6,b
7,b
8,b
9,a
